In [1]:
# Import modules

import random

from IPython.display import clear_output


# Create variables

players = []
playing_game = 1 # To end game 
suits = ('\u2660','\u2665','\u2666','\u2663') # spade, heart, diamond, club
ranks = ('2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A')
values = {'2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, '10':10, 'J':10,
         'Q':10, 'K':10, 'A':11}
dummy = ['┌───────┐','|       |','|       |','|   ?   |','|       |','|       |','└───────┘'] # Closed card for dealer



# Create object classes

class Card:
    
    # Each card has a rank, suit, class, and a list of strings used to print the card
    def __init__(self,rank,suit):
        self.rank = rank
        self.suit = suit
        self.value = values[self.rank]
        self.l = ['┌───────┐',f'| {self.rank:<2}    |','|       |',f'|   {self.suit}   |',
                  '|       |',f'|    {self.rank:>2} |','└───────┘']
    
    
    # To print individual card. Not used in gameplay.
    def __str__(self):
        return f'{self.l[0]}\n{self.l[1]}\n{self.l[2]}\n{self.l[3]}\n{self.l[4]}\n{self.l[5]}\n{self.l[6]}'
    
    
    
class Deck:
    
    def __init__(self,num=1):
        self.deck = []  # Start with an empty list
        self.num = num
        # Create all the cards and append them to the list
        for n in range(num):
            self.add_deck()
        self.shuffle()
                
    def add_deck(self):
        for suit in suits:
            for rank in ranks:
                self.deck.append(Card(rank,suit))
    
    def shuffle(self):
        random.shuffle(self.deck)
    
    # Not used, but in case there's ever a need to check the whole deck..
    """def __str__(self):
        deck = 'Current deck:\n'
        for card in self.deck:
            deck += f'\n{card}'
        return deck"""
    
    


        
class Hand:
    def __init__(self, show=1):
        self.cards = []      # Will be a list of cards. 
        self.show = show     # Default is to show. Dealer set with Hand(0).
        self.value = 0       # Default value of cards.
        self.ace_counter = 0 # Added to in add_card(), removed in adjust_for_ace().
        self.playing = 1     # Each hand is playing to start, switched off when an end-game condition is fulfilled.
        self.bet = 0         # Default bet.
        self.split = 0       # To keep track of split history
        
        # The following variables represent end-game conditions:
        self.blackjack = 0
        self.busted = 0
        self.result = 0
        self.stood = 0
        self.twentyone = 0
        self.hand_l = []
        
        # W/L/P
        self.won = 0
        self.lost = 0
        self.pushed = 0
    
    def add_card(self):
        # Pop card off deck and add it to current hand
        self.cards.append(deck.deck.pop())
        # Add card's value of hand
        self.value += self.cards[-1].value
        # Keep track of aces
        if self.cards[-1].rank == 'A':
            self.ace_counter += 1
        else:
            pass
        self.make_list()
        
        
    
    # If player hand passes 21, this checks for aces and reduces value accordingly, 
    # keeping track of aces that have been subtracted
    def adjust_for_ace(self):
        while self.value > 21 and self.ace_counter > 0:
            if self.ace_counter > 0:
                self.value -= 10
                self.ace_counter -= 1
            else:
                pass
            
    # Compile list to be used for printing 'player' class        
    def make_list(self):
        self.hand_l = []                     # Reset list every time
        
        # Open hands:
        if self.show:
            for line in range(7):
                txt = ''
                i = 0                               # We don't know how many cards each open hand has,
                while i < len(self.cards):          # so this 'i' counter tells us to put spaces between
                    txt += self.cards[i].l[line]   # strings until we reach the last card,
                    if i == len(self.cards) - 1:    # where we start new line instead.
                        pass
                    else:
                        txt += " "
                    i += 1
                self.hand_l.append(txt)
                    
        # Closed hands:
        else:
            for line in range(7): # Each card has 7 lines, hence the 7 iterations.
                txt = ''
                txt += self.cards[0].l[line] # Append a line from the open card,
                txt += " "                   # a space,
                txt += dummy[line]           # and a line from the closed card.
                self.hand_l.append(txt)
            
    # To print individual hand. Only dealer uses in gameplay (show = 0). Superseded by 'player' class.
    def __str__(self):
        
        hand = 'Dealer\n' # b/c we're only using this to display the dealer's hand
        
        # To display closed hands:
        if self.show == 0:
            for line in range(7): # Each card has 7 lines, hence the 7 iterations.
                hand += self.cards[0].l[line] # Append a line from the open card,
                hand += " "                   # a space,
                hand += dummy[line]           # and a line from the closed card.
                if line != 6:
                    hand += "\n"              # Start a new line, except after the last line.
                else: 
                    pass
                    
        # To display open hands:
        else:
            for line in range(7):
                i = 0                               # We don't know how many cards each open hand has,
                while i < len(self.cards):          # so this 'i' counter tells us to put spaces between
                    hand += self.cards[i].l[line]   # strings until we reach the last card,
                    if i != len(self.cards) - 1:    # where we start new line instead.
                        hand += " "
                    else:
                        if line != 6:               # ...unless we've reached the last (7th) line.
                            hand += "\n"
                        else:
                            pass
                    i += 1
                    
        return hand


    
class Player:
    
    def __init__(self, name):
        self.hands = []      # Will be a list of hands.
        self.name = name     # Player name declared.
        self.stack = 0
    
    def add_hand(self):
        hand = Hand()
        hand.add_card()
        hand.add_card()
        self.hands.append(hand)
        
    def add_chips(self,amt):        # Used to increase stack when buying chips or winning hands.
        self.stack += amt
    
    def lose_chips(self, bet):      # Decrease stack when player loses.
        self.stack -= bet
    
    def __str__(self):
        
        player = f"{self.name} - ${self.stack}\n"     # Print player name and current stack
        
        # Print hand number above each of player's hands and current bets
        for i in range(len(self.hands)):
            string = f"Hand {i + 1}, wager: ${self.hands[i].bet}"
            player += f"{string:<{len(self.hands[i].hand_l[0])}}    "  # Left justified with spaces until start of next hand (in case of split).
        player += "\n"
        
        # Print the cards
        for line in range(7):         # Iterate through hand lists adding to string
            i = 0
            while i < len(self.hands):
                player += self.hands[i].hand_l[line]
                if i == len(self.hands):
                    pass
                else:
                    player += "    " # Extra space between hands
                i +=1
            if line == 6:
                pass
            else:
                player += "\n"       # New lines except after 7th
        
        # Print results
        bjs = 0
        for i in range(len(self.hands)):
            if self.hands[i].blackjack:
                bjs += 1
        for i in range(len(self.hands)):
            result = ""
            if i == 0:
                result += "\n"
            if self.hands[i].blackjack and not dealer.blackjack:
                result += "Blackjack!"
                player += f"{result:<{len(self.hands[i].hand_l[0])}}     "
            elif self.hands[i].twentyone and self.hands[i].won:
                result += "21!"
                player += f"{result:<{len(self.hands[i].hand_l[0])}}     "
            elif self.hands[i].busted:
                result += "Busted!"
                player += f"{result:<{len(self.hands[i].hand_l[0])}}     "
            elif self.hands[i].pushed:
                result += "Push."
                player += f"{result:<{len(self.hands[i].hand_l[0])}}     "
            elif self.hands[i].won:
                result += "Congratulations!"
                player += f"{result:<{len(self.hands[i].hand_l[0])}}     "
            elif self.hands[i].lost:
                result += "Too bad!"
                player += f"{result:<{len(self.hands[i].hand_l[0])}}     "
            elif bjs:
                result += " "
                player += f"{result:<{len(self.hands[i].hand_l[0])}}     "
            else:
                pass
        
        for i in range(len(self.hands)):
            wl = ""
            if i == 0:
                wl += "\n"
            if self.hands[i].blackjack and not dealer.blackjack:
                wl += f"You won ${self.hands[i].bet * 2}!"
                player += f"{wl:<{len(self.hands[i].hand_l[0])}}     "
            elif self.hands[i].twentyone and self.hands[i].won:
                wl += f"You won ${self.hands[i].bet}!"
                player += f"{wl:<{len(self.hands[i].hand_l[0])}}     "
            elif self.hands[i].busted:
                wl += f"You lost ${self.hands[i].bet}."
                player += f"{wl:<{len(self.hands[i].hand_l[0])}}     " 
            elif self.hands[i].pushed:
                wl += " "
                player += f"{wl:<{len(self.hands[i].hand_l[0])}}     "
            elif self.hands[i].won:
                wl += f"You won ${self.hands[i].bet}!"
                player += f"{wl:<{len(self.hands[i].hand_l[0])}}     " 
            elif self.hands[i].lost:
                wl += f"You lost ${self.hands[i].bet}."
                player += f"{wl:<{len(self.hands[i].hand_l[0])}}     "
            elif bjs:
                wl += " "
                player += f"{wl:<{len(self.hands[i].hand_l[0])}}     "
            else:
                pass
        
        return player
                    
    
    
# Functions

# Called at the beginning of the game and if player runs out of chips.
def add_funds(active_player):
    buy = input(f"{active_player.name}, would you like to buy some chips? ")
    if buy.capitalize() in ("Yes","Y"):
        money = int(input("How many would you like? $"))
        active_player.add_chips(money)
    elif buy.capitalize() in ("No","N"):                    # If player doesn't want to buy chips,
        if active_player.stack <= 0:                               # check if they have any.
            remove_player(active_player)
            print("Thanks for playing. Come again soon!")   # If they don't, end game.
        else:
            print("Okay, have fun playing.")                # If they do, continue.
    else:
        print("Please respond 'yes' or 'no.'")              # In case of bad input.
        add_funds(active_player)


# Called at the beginning of each round.
def take_bets():
    global players
    for p in players:
        if p.hands[0].bet == 0:
            if p.stack > 0:
                try:
                    bet = int(input(f"{p.name}, You currently have ${p.stack}. How much would you like to bet? $"))
                except ValueError:
                    print("I'm sorry, I can only accept numerical values.")             # Check for integer.
                else:
                    if bet <= p.stack:
                        p.hands[0].bet = bet                                           # Bet applied to hand's 'bet' variable
                    else:
                        print("I'm sorry. It looks like you're a little low on chips.") # If player tries to bet more than they have,
                        print(f"You currently have  ${p.stack}.")
                        choice = input("Would you like to buy more chips?" )            # give option to buy more,
                        if choice.capitalize() in ("Yes","Y"):
                            buy = int(input("How many would you like? "))
                            p.add_chips(buy)
                            take_bets()
                        elif choice.capitalize() in ("No","N"):                           # or they can make a lower bet.
                            take_bets()
                        else:
                            pass                                                        # add try/except statement?
            else:
                pass
        else:
            pass
        

# Find out how many and the names of the players.      
def def_players():
    global players
    try:
        num = int(input("How many players will we have today? "))
    except:
        print("I'm sorry, I can only accept numerical values.")
        deal()
    else:
        for n in range(num):
            players += [Player(input(f"What is your name, player {n+1}? ").capitalize())]    # Create list of 'player' class objects
    finally:
        pass

    
# A little weird, b/c I deal before taking bets, but I haven't showed them the cards yet, so I guess it works.
def deal():
    global players
    global dealer
    global deck
    
    # Reinitialize dealer hand
    dealer = Hand(0)
    
    # Reinitialize deck with 2 decks
    deck = Deck(2)
    
    # Clear dealer hand if it's not already empty
    i = 0
    while i < len(dealer.cards):
        dealer.cards.pop()
    
    dealer.add_card()
    dealer.add_card()
    
    # Clear player hands
    for p in players:
        p.hands.clear()
    
    for p in players:
        p.add_hand()   # Creates a 'hand' class object for each player and adds 2 cards

    
    
    
def check_blackjack():
    if dealer.value == 21:
        dealer.blackjack = 1
        dealer.show = 1
        for p in players:
            for h in p.hands:
                h.playing = 0
    else:
        pass
    for p in players:
        for h in p.hands:
            if h.value == 21:
                h.blackjack = 1 
                h.playing = 0
            else:
                pass


def show_table():
    clear_output()
    print(dealer)
    for p in players:
        print("\n")
        print(p)
        


def choices():
    global players
    while turnmaster():
        for p in players:
            i = 0
            while i < len(p.hands):
                if p.hands[i].playing:
                    print(f"\nPlaying: {p.name}, Hand {i + 1}")

                    can_double = 0
                    sum_bets = 0
                    can_split = 0
                    for h in p.hands:
                        sum_bets += h.bet
                    if sum_bets + p.hands[i].bet <= p.stack:
                        can_double = 1
                        can_split = 1
                    else:
                        pass
                    if len(p.hands[i].cards) > 2:
                        can_double = 0
                    else: 
                        pass
                    if p.hands[i].cards[0].value != p.hands[i].cards[1].value or p.hands[i].split > 1:
                        can_split = 0
                    else:
                        pass

                    if can_double and can_split:
                        choice = input("Hit, Stand, Double, or Split?")
                        if choice.capitalize() in ("Hit", "H"):
                            p.hands[i].add_card()
                            check_twentyone_bust(p.hands[i])
                            show_table()
                        elif choice.capitalize() in ("Stand","S","St","Stnd"):
                            p.hands[i].stood = 1
                            p.hands[i].playing = 0
                            show_table()
                        elif choice.capitalize() in ("Double","D","Dble","Dub"):
                            p.hands[i].bet *= 2
                            p.hands[i].playing = 0
                            p.hands[i].add_card()
                            check_twentyone_bust(p.hands[i])
                            show_table()
                        elif choice.capitalize() in ("Split","Sp","Spl","Splt"):
                            p.add_hand()

                            p.hands[i].value -= p.hands[i].cards[-1].value       # Subtract value of cards about to be popped
                            p.hands[-1].value -= p.hands[-1].cards[-1].value

                            p.hands[-1].cards.insert(0, p.hands[i].cards.pop())  # Pop card from old hand and add it to front of new hand
                            p.hands[-1].value += p.hands[-1].cards[0].value      # Add value of card just added to new hand

                            p.hands[-1].cards.pop()                              # Remove extra card from new hand
                            
                            p.hands[-1].bet = p.hands[i].bet                     # Make bet of new hand equal to original hand
                            p.hands[i].split += 1
                            p.hands[-1].split = p.hands[i].split                 # Keep track of split history by adding one to the split counter
                            
                            p.hands[-1].make_list()                              # Rewrite hand string
                            p.hands[i].add_card()                                
                            check_blackjack()
                            check_twentyone_bust(p.hands[i])
                            show_table()
                            continue
                        else:
                            print("Come again?")
                            continue
                    elif can_double:
                        choice = input("Hit, Stand, or Double?")
                        if choice.capitalize() in ("Hit", "H"):
                            p.hands[i].add_card()
                            check_twentyone_bust(p.hands[i])
                            show_table()
                        elif choice.capitalize() in ("Stand","S","St","Stnd"):
                            p.hands[i].stood = 1
                            p.hands[i].playing = 0
                            show_table()
                        elif choice.capitalize() in ("Double","D","Dble","Dub"):
                            p.hands[i].bet *= 2
                            p.hands[i].playing = 0
                            p.hands[i].add_card()
                            check_twentyone_bust(p.hands[i])
                            show_table()
                        else:
                            print("Come again?")
                            continue
                    elif can_split:
                        choice = input("Hit, Stand, or Split?")
                        if choice.capitalize() in ("Hit", "H"):
                            p.hands[i].add_card()
                            check_twentyone_bust(p.hands[i])
                            show_table()
                        elif choice.capitalize() in ("Stand","S","St","Stnd"):
                            p.hands[i].stood = 1
                            p.hands[i].playing = 0
                            show_table()
                        elif choice.capitalize() in ("Split","Sp","Spl","Splt"):
                            p.add_hand()

                            p.hands[i].value -= p.hands[i].cards[-1].value       # Subtract value of cards about to be popped
                            p.hands[-1].value -= p.hands[-1].cards[-1].value

                            p.hands[-1].cards.insert(0, p.hands[i].cards.pop())  # Pop card from old hand and add it to front of new hand
                            p.hands[-1].value += p.hands[-1].cards[0].value      # Add value of card just added to new hand

                            p.hands[-1].cards.pop()                              # Remove extra card from new hand
                            
                            p.hands[-1].bet = p.hands[i].bet                     # Make bet of new hand equal to original hand
                            p.hands[i].split += 1
                            p.hands[-1].split = p.hands[i].split                 # Keep track of split history by adding one to the split counter
                            
                            p.hands[-1].make_list()                              # Rewrite hand string
                            p.hands[i].add_card() 
                            check_blackjack()
                            check_twentyone_bust(p.hands[i])
                            show_table()
                            continue
                        else:
                            print("Come again?")
                            continue
                    else:
                        choice = input("Hit or Stand?")
                        if choice.capitalize() in ("Hit", "H"):
                            p.hands[i].add_card()
                            check_twentyone_bust(p.hands[i])
                            show_table()
                        elif choice.capitalize() in ("Stand","S","St","Stnd"):
                            p.hands[i].stood = 1
                            p.hands[i].playing = 0
                        else:
                            print("Come again?")
                            continue
                else:
                    pass
                i += 1

            
def check_twentyone_bust(active_hand):
    if active_hand.value == 21 and not active_hand.blackjack:
        active_hand.twentyone = 1
        active_hand.playing = 0
    elif active_hand.value > 21:
        active_hand.adjust_for_ace()
        if active_hand.value == 21:
            active_hand.twentyone = 1
            active_hand.playing = 0
        elif active_hand.value > 21: 
            active_hand.busted = 1
            active_hand.playing = 0
            
    else:
        pass
    
    

def turnmaster():
    active_hands = 0
    for p in players:
        for h in p.hands:
            active_hands += h.playing
    return active_hands

        

def dealer_moves():
    b_counter = 0
    for p in players:
        for h in p.hands:
            if h.busted or h.blackjack:
                b_counter += 1
            else:
                pass
    if b_counter:
        pass
    else:
        while dealer.value < 17:
            dealer.add_card()
            if dealer.value > 21:
                dealer.adjust_for_ace()
    dealer.show = 1
        
def check_results():
    for p in players:
        for h in p.hands:
            if h.busted:
                h.lost = 1
            elif h.value == dealer.value:
                h.pushed = 1
            elif h.blackjack:
                h.won = 1
            elif h.twentyone:
                h.won = 1
            elif h.value > dealer.value:
                h.won = 1
            elif dealer.value > 21:
                h.won = 1
            else:
                h.lost = 1


    
    
def wins_and_losses():
    for p in players:
        for h in p.hands:
            if h.blackjack and h.won:
                p.stack += h.bet * 2
            elif h.won:
                p.stack += h.bet
            elif h.lost:
                p.stack -= h.bet
            else:
                print("Something went wrong here")
    
        
def another_round():
    global playing
    for p in players:
        choice = input(f"{p.name}, would you like to play another round?")
        try:
            choice.capitalize() in ("Yes","No","Y","N")
        except:
            print("Sorry, what was that?")
        else:
            if choice.capitalize() in ("Yes","Y"):
                if p.stack > 0:
                    choice2 = input("Would you like to stock up on chips first?")
                    if choice2.capitalize() in ("Yes","Y"):
                        money = int(input("How many would you like? $"))
                        p.add_chips(money)
                    elif choice2.capitalize() in ("No","N"):
                        print("Okay, enjoy playing!")
                    else:
                        pass
                else:
                    print("You need more chips to play")
                    money = int(input("How many would you like? $"))
                    if money > 0:
                        p.add_chips(money)
                    else:
                        print("Have a good evening.")
                        remove_player(p)
                    show_table()
            else:
                print("Thanks for playing, come back soon!")
                remove_player(p)

def remove_player(player):
    global players
    players = [p for p in players if p != player]
    
def joiners():
    j = input("Would anyone like to join for the next round? ")
    if j.capitalize() in ("Yes","Y"):
        join = int(input("How many players will be joining us? "))
        i = 0
        while i < join:
            name = input(f"What is your name, new player {join}? ")
            add_player(name)
            i += 1
    elif j.capitalize() in ("No","N"):
        pass
    else:
        print("I didn't understand")
        joiners()
    
def add_player(name):
    players.append(Player(name))
    add_funds(players[-1])
    
def sign_off():
    clear_output()
    print("Thanks for playing everyone. Have a great evening and come back any time!")

dealer = Hand(0)
deck = Deck(2)
                
def blackjack():
    def_players()
    for p in players:
        add_funds(p)
    while len(players):
        deck = Deck(2)     # Variable controls how many decks you're playing with
        deal()
        take_bets()
        check_blackjack()
        show_table()
        choices()
        dealer_moves()
        check_results()
        wins_and_losses()
        show_table()
        another_round()
        joiners()
    sign_off()

In [2]:
blackjack()

Thanks for playing everyone. Have a great evening and come back any time!
